In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from itertools import chain

# Importations
import seaborn as sns
import torch
from torch import nn
from torch.utils.data import DataLoader, BatchSampler, random_split

from src.covid_cytof_dataset import CovidCytofDataset
from src.models.configurable_nn import ConfigurableNeuralNetwork
from src.models.simple_nn import SimpleNeuralNetwork
from src.trainer import Trainer

sns.set_theme()

In [3]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
# device = "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
NUM_CELLS_PER_PATIENS = 1000
BATCH_SIZE = 100

In [5]:
data = CovidCytofDataset("./data/attachments/COVID_CYTOF_BASIC_METADATA_STATUS_AGE_GROUP_SEX.xlsx", "./data",
                         NUM_CELLS_PER_PATIENS)

/home/evann/.cache/pypoetry/virtualenvs/covid-cytof-NoJZp_V7-py3.10/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Loading fcs data:


100%|██████████| 99/99 [00:20<00:00,  4.84it/s]

done


In [6]:
train_data, test_data = random_split(list(BatchSampler(data, batch_size=NUM_CELLS_PER_PATIENS, drop_last=False)),
                                     [0.8, 0.2])
train_data, test_data = list(chain.from_iterable(train_data)), list(chain.from_iterable(test_data))

train_dataloader = DataLoader(train_data, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)

In [7]:
model = SimpleNeuralNetwork(data.data.shape[1], 500, 1).to(device)
print(model)

SimpleNeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=61, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=1, bias=True)
    (3): Sigmoid()
  )
)


In [8]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

trainer = Trainer(model, optimizer, loss_fn, device, epochs=10)

In [ ]:
trainer.run(train_dataloader, test_dataloader)

In [ ]:
trainer.plot_loss()
trainer.plot_accuracy()

In [ ]:
model1 = ConfigurableNeuralNetwork([
    data.data.shape[1],
    1
])

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

trainer = Trainer(model, optimizer, loss_fn, device, epochs=10)

trainer.run(train_dataloader, test_dataloader)

trainer.plot_loss()
trainer.plot_accuracy()

In [ ]:
model1 = ConfigurableNeuralNetwork([
    data.data.shape[1],
    100,
    1
])

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

trainer = Trainer(model, optimizer, loss_fn, device, epochs=10)

trainer.run(train_dataloader, test_dataloader)

trainer.plot_loss()
trainer.plot_accuracy()